In [1]:
import h5py
import numpy as np
import sys
import os
sys.path.append(os.path.abspath(".."))
from src.text_data import NoStreamingDataset
from tqdm import tqdm


/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [5]:
h5file = "../runs/test/mlm_efficiency/processed_samples/processed_samples_peptide.hdf5"
with h5py.File(h5file) as f:
	print (list(f.keys()))
	print (all(f['0'][:]))

['0', '1', '10', '100', '1000', '1001', '1002', '1003', '1004', '1005', '1006', '1007', '1008', '1009', '101', '1010', '1011', '1012', '1013', '1014', '1015', '1016', '1017', '1018', '1019', '102', '1020', '1021', '1022', '1023', '1024', '1025', '1026', '1027', '1028', '1029', '103', '1030', '1031', '1032', '1033', '1034', '1035', '1036', '1037', '1038', '1039', '104', '1040', '1041', '1042', '1043', '1044', '1045', '1046', '1047', '1048', '1049', '105', '1050', '1051', '1052', '1053', '1054', '1055', '1056', '1057', '1058', '1059', '106', '1060', '1061', '1062', '1063', '1064', '1065', '1066', '1067', '1068', '1069', '107', '1070', '1071', '1072', '1073', '1074', '1075', '1076', '1077', '1078', '1079', '108', '1080', '1081', '1082', '1083', '1084', '1085', '1086', '1087', '1088', '1089', '109', '1090', '1091', '1092', '1093', '1094', '1095', '1096', '1097', '1098', '1099', '11', '110', '1100', '1101', '1102', '1103', '1104', '1105', '1106', '1107', '1108', '1109', '111', '1110', '1111

In [ ]:

class _GenomeDatasetForMasking(NoStreamingDataset):
	def subsample_to_shards_slice(self, start, end):
		self.start_id = self.spanner.shard_bounds[start] # start is inclusive
		self.end_id = self.spanner.shard_bounds[end] # end is exclusive

data_dir = "/mnt/nfs_dna/shadskiy/promoters/pretrena/mds_v2/"

_dataset = _GenomeDatasetForMasking(
		local=data_dir,
		split="train",
		max_seq_len=1024,
		tokenizer=None,
)
total_samples = _dataset.spanner.num_samples
total_shards = len(_dataset.spanner.shard_sizes)
assert len(_dataset.spanner.shard_bounds) == total_shards + 1

/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:958: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/flash_attn/ops/triton/layer_norm.py:1017: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [9]:
total_samples, total_shards

(13525034, 5279)

In [13]:
# 0 - shard not processed, 
# 1 - shard processed, 2 - shard processed but not all samples processed
check = np.zeros(shape=(total_shards,), dtype=np.int8)
nodes = ["peptide", "olig", "mlspace"]
progress_files_loc = "../runs/test/mlm_efficiency/processed_samples"
for node in nodes:
	h5file = f"{progress_files_loc}/processed_samples_{node}.hdf5"
	with h5py.File(h5file) as f:
		for k in tqdm(f.keys()):
			if not all(f[k][:]):
				check[int(k)] = 2
			else:
				check[int(k)] = 1


100%|██████████| 3000/3000 [00:00<00:00, 3042.04it/s]


In [14]:
np.bincount(check)

array([   0, 5279])

In [2]:
from omegaconf import OmegaConf as om
from main import build_dataloader, build_model

import os
os.chdir(os.path.expanduser("~/DNALM/ModernBERT/"))

cfg = "yamls/moderngena/gena-large-pretrain_multi_promoters_v2_MLMfromEP39.yaml"
cfg = om.load(cfg)

model = build_model(cfg.model)

train_loader = build_dataloader(
	cfg=cfg.train_loader,
	tokenizer=model.tokenizer,
	device_batch_size=cfg.global_train_batch_size,
	count_padding_tokens=cfg.get("count_padding_tokens", True),
	device_microbatch_size=cfg.device_train_microbatch_size,
)

eval_loader = build_dataloader(
	cfg=cfg.eval_loader,
	tokenizer=model.tokenizer,
	device_batch_size=cfg.get("device_eval_batch_size"),
)

/disk/10tb/home/fishman/miniconda3/envs/bert24/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [17]:
%%time
for i in range(4604):
	train_loader.dataloader.dataset.__getitem__(i)

fetch 0, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 0, Done! 0:00:00.018085
fetch 1, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 1, Done! 0:00:00.019005
fetch 2, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 2, Done! 0:00:00.020034
fetch 3, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 3, Done! 0:00:00.013895
fetch 4, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 4, Done! 0:00:00.025153
fetch 5, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0.hdf5
fetch 5, Done! 0:00:00.025798
fetch 6, data/mlm_acc_from_base_ep129/train/
Acquiring mlm probs for  data/mlm_acc_from_base_ep129/train/shard_0

In [ ]:
%%time
for i in range(4604):
	eval_loader.dataloader.dataset.__getitem__(i)

fetch 0, None
fetch 0, Done! 0:00:00.012129
fetch 1, None
fetch 1, Done! 0:00:00.006994
fetch 2, None
fetch 2, Done! 0:00:00.007187
fetch 3, None
fetch 3, Done! 0:00:00.007392
fetch 4, None
fetch 4, Done! 0:00:00.008163
fetch 5, None
fetch 5, Done! 0:00:00.007185
fetch 6, None
fetch 6, Done! 0:00:00.007983
fetch 7, None
fetch 7, Done! 0:00:00.008911
fetch 8, None
fetch 8, Done! 0:00:00.009335
fetch 9, None
fetch 9, Done! 0:00:00.008514
CPU times: user 20.7 ms, sys: 4.45 ms, total: 25.1 ms
Wall time: 85.5 ms


dict_keys(['chunk_offset', 'file_id', 'line_id', 'text'])

In [6]:
import argparse
import json
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from streaming import MDSWriter
from tqdm.auto import tqdm

In [ ]:
columns = {
	"text": "str",
	"file_id": "str",
	"line_id": "int",
	"chunk_offset": "int",
	"MLM": "ndarray:uint8"
}

full_out_dir = "data/promoters_mds_v2_with_mlm_larger/train/"
!rm -r {full_out_dir}
os.makedirs(full_out_dir, exist_ok=True)

ds = train_loader.dataloader.dataset

with MDSWriter(columns=columns, out=full_out_dir, size_limit='64mb') as fout:
	# for sample_id in tqdm(range(len(ds))):
	for sample_id in tqdm(range(len(ds))):	
		shard_id, shard_sample_id = ds.spanner[sample_id]
		shard = ds.shards[shard_id]
		sample = shard[shard_sample_id]
		hdf5_file = os.path.join(ds.mlm_efficiency_path, f"shard_{shard_id}.hdf5")
		with h5py.File(hdf5_file, "r") as f:
			mlm_probs = f[str(shard_sample_id)][:]
			# print (mlm_probs, np.mean(mlm_probs))
			mlm_probs = mlm_probs.astype(np.uint8)
			# print (mlm_probs, np.mean(mlm_probs))
			if sample_id>1_000_000:
				break
		sample["MLM"] = mlm_probs
		fout.write(sample)

rm: cannot remove 'data/promoters_mds_v2_with_mlm_larger/train/': No such file or directory


  4%|▍         | 587816/13525034 [1:14:27<25:08:20, 142.95it/s]